In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import seaborn as sns
import torch

In [46]:
def read_tif_to_np(tif_path):
    """Reads a tif file and converts it into a numpy.ndarray
    
    Arg:
        tif_path: The full path to the tif file to read.
    
    Returns:
        A"""
    with rasterio.open(y_tile_path) as f:
        data = np.rollaxis(f.read(), 0, 3)
        return data
    
def get_tile_y(base_dir, tile):
    """Retrieves the landcover y data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray containing the y data for the tile.
    
    """
    y_dir_ext = "_lc.tif"
    y_tile_path = base_dir + tile + y_dir_ext
    return read_tif_to_np(y_tile_path)

def get_tile_x(base_dir, tile):
    """Retrieves the landcover x data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray) containing the x data for the tile.
    
    """
    x_dir_ext = ".tif" 
    x_tile_path = base_dir + tile + x_dir_ext
    return read_tif_to_np(y_tile_path)

def apply_label_remap(labels, label_map):
    """Reassigns labels inplace in an numpy array given a provided mapping.
    
    Args:
        labels: An ndarray of labels.
        label_map: A dict[int, int] mapping label classes [original, new].
        
    """
    for l1, l2 in label_map.items():
        labels[labels == l1] = l2


# Define tiles to be used in testing, training and finetuning.
tile_dir_y = "/mnt/blobfuse/resampled-lc/data/v1/2015/states/md/md_1m_2015/39076/"
tile_dir_x = "/mnt/blobfuse/esri-naip/v002/md/2015/md_100cm_2015/39076/" 
tile = "m_3907639_sw_18_1_20150815"
neighbor_tile = "m_3907639_ne_18_1_20150815"

# Read X and Y data for the tile and apply label transformations.
x = get_tile_x(tile_dir_x, neighbor_tile)
y = get_tile_y(tile_dir_y, neighbor_tile)

water_forest_land_impervious_remap = {5: 4, 6: 4}
apply_label_remap(y, water_forest_land_impervious_remap)



In [37]:
x_dir = "esri-naip"
x_dir_ext = ".mrf" 
x_tile_path = base_dir.format(data_dir=x_dir) + tile + x_dir_ext

In [38]:
x_tile_path

'/mnt/blobfuse/esri-naip/data/v1/2015/states/md/md_1m_2015/39076/m_3907639_sw_18_1_20150815.mrf'

In [42]:
with open(x_tile_path, "rb") as f:
    print (f)

<_io.BufferedReader name='/mnt/blobfuse/esri-naip/data/v1/2015/states/md/md_1m_2015/39076/m_3907639_sw_18_1_20150815.mrf'>


In [48]:
x.shape

(7606, 6066)

In [49]:
y.shape

(7606, 6066, 1)